Постановка задачи:
$$\underset{x \in S}{min} \; f(x) = \frac{1}{2} (x, Ax) + (b, x)$$
$$S = \{ x \in \mathbb{R}^n : (x,x) = 1 \}$$

Поскольку аналитически решение такой задачи получить не представляется возможным, решим ее методом сопряженных градиентов:
\begin{equation} 
x^{k+1} = P_S(x^k - \alpha_k p^k) = \frac{x^k - \alpha_k p^k}{\| x^k - \alpha_k p^k \|},
\end{equation}
где $\alpha_k$, $p^k$ определяются следующим образом:
\begin{equation}
\begin{aligned}
\alpha_k &= \underset{\alpha}{argmin} \; f(P_S(x^k + \alpha p^k))  = \\
&= \underset{\alpha}{argmin}\Bigg\{ \frac{1}{2} \frac{(x^k + \alpha p^k, A(x^k + \alpha p^k))}{\| x^k + \alpha p^k \|^2} + \frac{(b, x^k + \alpha p^k)}{\| x^k + \alpha p^k \|} \Bigg\},\; x_0 \in S \\
p^k &= P_S(-\nabla f(x^k) - \beta_k p^{k-1}) = \frac{-(Ax^k + b) - \beta_k p^{k-1}}{\| -(Ax^k + b) - \beta_k p^{k-1} \|}, \; p^0 = -\frac{Ax^0 + b}{\| Ax^0 + b \|}, \\
\beta_k &= \frac{\| P_S(\nabla f(x^k)) \|^2}{\| P_S(\nabla f(x^{k-1}))\|^2} = 1
\end{aligned}
\end{equation}

In [3]:
import numpy as np

In [4]:
A = np.array([[1., 0., 2.], [0., 2., 3.], [2., 3., 5.]])
b = np.array([0., 5., 13.])
print A
print b

[[ 1.  0.  2.]
 [ 0.  2.  3.]
 [ 2.  3.  5.]]
[  0.   5.  13.]


In [7]:
x0 = np.array([1., 1., 1.])
x0 = x0 / np.linalg.norm(x0)
p0 = - (np.dot(A, x0) + b) / (np.linalg.norm(np.dot(A, x0) + b))

In [9]:
def getXkk(xk, alphak, pk):
    return (xk - alphak * pk) / np.linalg.norm(xk - alphak * pk)
def getPkk(pk, xk):
    return (np.dot(-A, xk) - b - pk) / np.linalg.norm(np.dot(-A, xk) - b - pk)
def gradAlpha(xk, pk, alpha):
    first = ((xk + alpha * pk).dot(np.dot(A, pk)) * np.linalg.norm(xk + alpha * pk) ** 2 - 2 * np.dot(xk + alpha * pk, pk) * (xk + alpha * pk).dot(np.dot(A, xk + alpha * pk))) / (np.linalg.norm(xk + alpha * pk) ** 4)
    second = (np.dot(b, pk) * np.linalg.norm(xk + alpha * pk) - (np.dot(xk + alpha * pk, pk))/(np.linalg.norm(xk + alpha * pk)) * np.dot(b, xk + alpha * pk)) / (np.linalg.norm(xk + alpha * pk) ** 2)
    return first + second

Для вычисления шага $\alpha_k$ будем делать градиентный спуск с априорным выбором шага. Последовательность шагов будет такова, что ряд из шагов расходится, а из квадратов сходится.

In [76]:
def getNewAlpha(xk, pk, eps):
    alphak = 0.
    n = 1
    notEnough = True
    while (notEnough):
        if (1. / n) * gradAlpha(xk, pk, alphak) < eps:
            notEnough = False
        alphakk = alphak - (1. / n) * gradAlpha(xk, pk, alphak)
        alphak = alphakk
        n = n + 1
    return alphakk

In [77]:
#test cell
print getNewAlpha(x0, p0, 1e-5)

2.47518326397


In [78]:
def sphereCGM(x0, p0, eps, epsalpha):
    notEnough = True
    xk = x0
    pk = p0
    while (notEnough):
        alphak = getNewAlpha(xk, pk, epsalpha)
        xkk = getXkk(xk, alphak, pk)
        if np.linalg.norm(xkk - xk) < eps:
            notEnough = True
        pk = getPkk(xk, pk)
        xk = xkk
    return xk
        
        

In [79]:
print sphereCGM(x0, p0, 1e-5, 1e-5)

KeyboardInterrupt: 